In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install polars keras==2.15

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import backend as K
import tensorflow as tf

import os
import polars as pl
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

AUTO = tf.data.experimental.AUTOTUNE
print("REPLICAS: ", strategy.num_replicas_in_sync)

# Data PreProcessing

In [ ]:
%%time
train_df = pl.read_csv('/kaggle/input/leap-atmospheric-physics-ai-climsim/train.csv')
test_df = pl.read_csv('/kaggle/input/leap-atmospheric-physics-ai-climsim/test.csv')

### Separate the target and feature columns for earier analysis and computation in the training dataset.

In [ ]:
FEAT_COLS = train_df.columns[1:557]
TARGET_COLS = train_df.columns[557:]

### Cast from Float 64 to Float 32 for better memory efficiency. and convert to numpy

In [ ]:
for col in FEAT_COLS:
    train_df = train_df.with_columns(pl.col(col).cast(pl.Float32))
    test_df = test_df.with_columns(pl.col(col).cast(pl.Float32))

for col in TARGET_COLS:
    train_df = train_df.with_columns(pl.col(col).cast(pl.Float32))

In [ ]:
X_train = train_df.select(FEAT_COLS).to_numpy()
y = train_df.select(TARGET_COLS).to_numpy()
test_feats = test_df.select(FEAT_COLS).to_numpy()

### Scale data in a way where we apply the same scaling in the train data, to the test data, to avoid data leakage. Converts to numpy for faster compute

In [ ]:
# norm X
#sets a minimum standard deviation threshold to avoid division by zero.
min_std = 1e-8

mx = X_train.mean(axis=0)
sx = np.maximum(X_train.std(axis=0), min_std)
X_train = (X_train - mx.reshape(1,-1)) / sx.reshape(1,-1)
test_feats = (test_feats - mx.reshape(1,-1)) / sx.reshape(1,-1)

# norm Y
my = y.mean(axis=0)
sy = np.maximum(np.sqrt((y*y).mean(axis=0)), min_std)
y = (y - my.reshape(1,-1)) / sy.reshape(1,-1)

# Model Setup

In [ ]:
from tensorflow.keras.applications import EfficientNetV2B3

class CustomModel(tf.keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        
        self.efficient_net = EfficientNetV2B3(include_top=False, weights=None, input_shape=(556, 32, 1))
        self.global_avg_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.output_layer = tf.keras.layers.Dense(len(TARGET_COLS))
    
    def call(self, x):
        x = tf.expand_dims(x, axis=-1)  # Add channel dimension
        x = tf.tile(x, multiples=[1, 1, 32])  # Repeat the data along the second dimension to match (batch_size, 556, 32, 1)
        x = self.efficient_net(x)
        x = self.global_avg_pool(x)
        x = self.output_layer(x)
        return x

In [ ]:
# class CustomModel(tf.keras.Model):
#     def __init__(self):
#         super(CustomModel, self).__init__()
        
#         self.conv_layers = [
#             tf.keras.layers.Reshape((556, 1), input_shape=(556,)),
            
#             tf.keras.layers.Conv1D(64, 3, padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#             tf.keras.layers.BatchNormalization(),
#             tf.keras.layers.Activation('relu'),
#             tf.keras.layers.MaxPooling1D(2),
            
#             tf.keras.layers.Conv1D(128, 3, padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#             tf.keras.layers.BatchNormalization(),
#             tf.keras.layers.Activation('relu'),
#             tf.keras.layers.MaxPooling1D(2),
            
#             tf.keras.layers.Conv1D(256, 3, padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#             tf.keras.layers.BatchNormalization(),
#             tf.keras.layers.Activation('relu'),
#             tf.keras.layers.MaxPooling1D(2),
            
#             tf.keras.layers.Conv1D(512, 3, padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#             tf.keras.layers.BatchNormalization(),
#             tf.keras.layers.Activation('relu'),
#             tf.keras.layers.GlobalMaxPooling1D(),
            
#             tf.keras.layers.Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#             tf.keras.layers.BatchNormalization(),
#             tf.keras.layers.Activation('relu'),
#             tf.keras.layers.Dropout(0.5),
            
#             tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#             tf.keras.layers.BatchNormalization(),
#             tf.keras.layers.Activation('relu'),
#             tf.keras.layers.Dropout(0.3)
#         ]
        
#         self.output_layer = tf.keras.layers.Dense(len(TARGET_COLS))
    
#     def call(self, x):
#         for layer in self.conv_layers:
#             x = layer(x)
#         x = self.output_layer(x)
#         return x

In [ ]:
# class CustomModel(tf.keras.Model):
#     def __init__(self):
#         super(CustomModel, self).__init__()
        
#         self.hidden_sizes = [256, 512, 768, 1024, 768, 512, 256, 128, 64, 32, 16, 8]
#         self.dropout_rates = [0.2, 0.3, 0.4, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.025, 0.0125, 0.00625]
#         self.l2_reg = 0.0005
        
#         self.dense_layers = []
#         for hidden_size, dropout_rate in zip(self.hidden_sizes, self.dropout_rates):
#             self.dense_layers.append(tf.keras.layers.Dense(hidden_size, kernel_regularizer=tf.keras.regularizers.l2(self.l2_reg)))
#             self.dense_layers.append(tf.keras.layers.BatchNormalization())
#             self.dense_layers.append(tf.keras.layers.Activation('swish'))
#             self.dense_layers.append(tf.keras.layers.Dropout(dropout_rate))
        
#         self.output_layer = tf.keras.layers.Dense(len(TARGET_COLS))
    
#     def call(self, x):
#         skip_connections = []
#         for i, layer in enumerate(self.dense_layers):
#             if i % 8 == 0 and i != 0:  # Add skip connection every 8 layers (except the first layer)
#                 if skip_connections:
#                     x = tf.keras.layers.Concatenate()([x] + skip_connections)
#                     skip_connections = []
#             x = layer(x)
#             if i % 4 == 0:  # Save the output for skip connection every 4 layers
#                 skip_connections.append(x)
#         x = self.output_layer(x)
#         return x

In [ ]:
def build_model():
    model = CustomModel()
    return model

In [ ]:
epochs = 50
batch_size = 512*strategy.num_replicas_in_sync
lr_patience = 15
early_patience = 30
min_lr = 1e-7
lr_factor = 0.2

# Training and Inference

In [ ]:
# def progressive_learning(model, x_train, y_train, x_valid, y_valid, epochs, batch_size):
#     # Define the progressive learning settings
#     min_size = 128
#     max_size = 300
    
#     # Calculate the step size for image size
#     size_step = (max_size - min_size) // (epochs - 1)
    
#     # Compile the model
#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
#     model.compile(optimizer=optimizer, loss="mse")
    
#     # Training loop with progressive learning
#     for epoch in range(epochs):
#         # Calculate the current image size
#         current_size = min_size + size_step * epoch
        
#         # Reshape the input data to the current size
#         x_train_reshaped = tf.reshape(x_train, (-1, current_size, x_train.shape[-1] // current_size))
#         x_valid_reshaped = tf.reshape(x_valid, (-1, current_size, x_valid.shape[-1] // current_size))
        
#         # Train the model for one epoch
#         model.fit(x_train_reshaped, y_train, batch_size=batch_size, epochs=1, validation_data=(x_valid_reshaped, y_valid))
    
#     return model

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y)):
    x_train, x_valid = X_train[train_idx], X_train[val_idx]
    y_train, y_valid = y[train_idx], y[val_idx]
    
    print(x_train.shape, x_valid.shape)

    checkpoint_filepath = f"folds{fold}.weights.h5"
    
    K.clear_session()
    with strategy.scope():
        model = build_model()
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
    model.compile(optimizer=optimizer, loss="mse")
    
    monitor = "val_loss"
    sv = ModelCheckpoint(
            checkpoint_filepath, monitor=monitor, verbose=1, save_best_only=True,
            save_weights_only=True, mode='min', save_freq='epoch'
    )
    reduce_lr = ReduceLROnPlateau(monitor=monitor, factor=lr_factor,
                              patience=lr_patience, min_lr=min_lr)
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor=monitor,
        min_delta=0.0,
        patience=early_patience, 
        mode="min"
    )

    history = model.fit(x_train, y_train, verbose=1,
                        validation_data=(x_valid, y_valid), 
                        epochs=epochs, batch_size=batch_size, callbacks=[reduce_lr, sv, early_stop])
    
    oof_pred = model.predict(x_valid)    
    oof_true = y_valid

    test_pred = model.predict(test_feats)
    
    break

# Submission file

In [ ]:
# submit
# override constant columns
for i in range(sy.shape[0]):
    if sy[i] < min_std * 1.1:
        test_pred[:,i] = 0

# undo y scaling
test_pred = test_pred * sy.reshape(1,-1) + my.reshape(1,-1)

In [ ]:
sub = pd.read_csv("/kaggle/input/leap-atmospheric-physics-ai-climsim/sample_submission.csv")
sub.iloc[:,1:] *= test_pred

In [ ]:
test_polars = pl.from_pandas(sub[["sample_id"]+TARGET_COLS])
test_polars.write_csv("submission.csv")